In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stat

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, auc
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import graphviz

import scikitplot as skplt

from subprocess import check_output
import os

%matplotlib inline

sns.set_style('whitegrid')
sns.set_palette("Set1", n_colors=8, desat=.5)
# current_palette = sns.color_palette("Set1", n_colors=8, desat=.5)
# sns.palplot(current_palette)
print(check_output(["ls", "./"]).decode("utf8"))

Challenge 10.ipynb
cc_info.csv
transactions.csv



In [10]:
cc = pd.read_csv('./cc_info.csv')
df = pd.read_csv('./transactions.csv')

In [11]:
cc.head()

,credit_card,city,state,zipcode,credit_card_limit
0,1280981422329509,Dallas,PA,18612,6000
1,9737219864179988,Houston,PA,15342,16000
2,4749889059323202,Auburn,MA,1501,14000
3,9591503562024072,Orlando,WV,26412,18000
4,2095640259001271,New York,NY,10001,20000


In [12]:
df.head()

,credit_card,date,transaction_dollar_amount,Long,Lat
0,1003715054175576,2015-09-11 00:32:40,43.78,-80.174132,40.267370
1,1003715054175576,2015-10-24 22:23:08,103.15,-80.194240,40.180114
2,1003715054175576,2015-10-26 18:19:36,48.55,-80.211033,40.313004
3,1003715054175576,2015-10-22 19:41:10,136.18,-80.174138,40.290895
4,1003715054175576,2015-10-26 20:08:22,71.82,-80.238720,40.166719


In [13]:
df['date'] = pd.to_datetime(df.date)
df['month'] = df.date.dt.month
df['dow'] = df.date.dt.dayofweek
df['hour'] = df.date.dt.hour

In [14]:
# df = pd.merge(df, cc, on = 'credit_card', how = 'left')
# df.head()

In [18]:
data = df.groupby(['credit_card','month']).sum()['transaction_dollar_amount'].reset_index()

In [19]:
data = pd.merge(data, cc, on = 'credit_card', how = 'left')

,credit_card,month,transaction_dollar_amount,city,state,zipcode,credit_card_limit
0,1003715054175576,7,162.56,Houston,PA,15342,20000
1,1003715054175576,8,8441.51,Houston,PA,15342,20000
2,1003715054175576,9,11281.56,Houston,PA,15342,20000
3,1003715054175576,10,8954.21,Houston,PA,15342,20000
4,1013870087888817,7,281.43,Washington,NH,3280,15000


In [20]:
data['exceed'] = data['transaction_dollar_amount'] > data['credit_card_limit']
data.head()

,credit_card,month,transaction_dollar_amount,city,state,zipcode,credit_card_limit,exceed
0,1003715054175576,7,162.56,Houston,PA,15342,20000,False
1,1003715054175576,8,8441.51,Houston,PA,15342,20000,False
2,1003715054175576,9,11281.56,Houston,PA,15342,20000,False
3,1003715054175576,10,8954.21,Houston,PA,15342,20000,False
4,1013870087888817,7,281.43,Washington,NH,3280,15000,False


In [24]:
exceed = data.groupby('credit_card').max()['exceed'].reset_index()
exceed[exceed.exceed == False].credit_card

0      1003715054175576
1      1013870087888817
2      1023820165155391
3      1073931538936472
4      1077622576192810
5      1087468642191606
6      1089479586475600
7      1101164752077181
9      1107936587129946
10     1126571863414211
11     1128536627506856
12     1134742235676586
13     1137926374621536
14     1144894232607400
15     1155726765596507
16     1156998467006869
17     1165413984579076
18     1167475218806361
19     1169271395749970
22     1191702776174515
23     1210872963953886
24     1221421251374063
25     1222730774437945
26     1224132812823172
27     1228288249952859
29     1250832548098023
30     1254472912142815
31     1261411767321605
32     1264384810841217
34     1281918894315694
             ...       
951    9678367852258782
952    9681744051023898
953    9683778679967012
954    9697976691448340
955    9698400826549648
956    9711468735134956
957    9722420012449776
960    9750493074217554
961    9755306333757844
962    9757152469896436
963    975820364